In [17]:
!pip freeze

annotated-types==0.7.0
anyio==4.9.0
appnope==0.1.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.5
attrs==25.3.0
babel==2.17.0
beautifulsoup4==4.13.4
bleach==6.2.0
certifi==2025.1.31
cffi==1.17.1
charset-normalizer==3.4.1
comm==0.2.2
contourpy==1.3.1
cramjam==2.9.1
cycler==0.12.1
debugpy==1.8.13
decorator==5.2.1
defusedxml==0.7.1
distro==1.9.0
executing==2.2.0
fastjsonschema==2.21.1
fastparquet==2024.11.0
filelock==3.18.0
fonttools==4.56.0
fqdn==1.5.1
fsspec==2025.3.0
h11==0.14.0
httpcore==1.0.7
httpx==0.28.1
huggingface-hub==0.29.3
idna==3.10
ipykernel==6.29.5
ipython==9.0.2
ipython_pygments_lexers==1.1.1
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.2
Jinja2==3.1.6
jiter==0.9.0
joblib==1.4.2
json5==0.12.0
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter-events==0.12.0
jupyter-lsp==2.2.5
jupyter_client==8.6.3
jupyter_core==5.7.2
jupyter_server==2.15.0
jupyter_server_terminals==0.5.3
jupyterlab==4.4.0

In [14]:
import os
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display

# Initialize the OpenAI client
client = OpenAI()

# Check if API key is set
if 'OPENAI_API_KEY' not in os.environ:
    print('OPENAI_API_KEY environment variable is not set.')

# Define the model to use
MODEL = 'gpt-4o-mini' 

### Helper Functions

In [15]:
def send_message(messages, model=MODEL, temperature=0.7, max_tokens=1000):
    """
    Send a message to the OpenAI API and get a response.
    
    Args:
        messages (list): List of message dictionaries with 'role' and 'content' keys
        model (str): The model to use
        temperature (float): Controls randomness (0-1)
        max_tokens (int): Maximum number of tokens to generate
        
    Returns:
        str: The model's response
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f'Error: {str(e)}'

def format_conversation(messages):
    """
    Format the conversation history for display.
    
    Args:
        messages (list): List of message dictionaries with 'role' and 'content' keys
        
    Returns:
        str: Formatted conversation history
    """
    formatted = []
    for message in messages:
        if message['role'] == 'system':
            continue  # Skip system messages in the display
        role = 'You' if message['role'] == 'user' else 'Assistant'
        formatted.append(f'<b>{role}:</b> {message["content"]}')
    return '<br>'.join(formatted)

### Interface

In [16]:
# Initialize conversation with a default system message
conversation = [
    {
        'role': 'system',
        'content': "You are ChillBuddy — a Gen Z-friendly, emotionally intelligent AI mental health counselor trained in CBT and cognitive reframing. Your response will be sent directly to a patient in an instant messaging app, so it must be short (2-3 sentences), useful, and make the patient feel better. In your response, you must: Quickly detect and label the patient's primary emotion (e.g., sadness, anger, anxiety, numbness). Identify the most likely primary mental health condition (e.g., Bipolar Disorder, Depression, Anxiety, General Mental Illness, etc.). Suggest one potential differential diagnosis if applicable (e.g., Mild anxiety, Unipolar depression, etc.). Provide succinct, actionable CBT-based advice using cognitive reframing. Use a casual, relatable Gen Z tone with occasional slang and emojis. Engage in friendly, encouraging conversation. Internally generate and score multiple responses, but output only the best one. At the end of your response, include a final line formatted exactly as: \"Detected: [Emotion], Primary: [Primary Disorder], Differential: [Differential Diagnosis]\" Keep the entire output brief, supportive, and uplifting."
    }
]

# Create widgets
system_label = widgets.HTML(value='<b>System Message:</b>')
system_input = widgets.Textarea(
    value=conversation[0]['content'],
    placeholder='Enter system message here...',
    description='',
    disabled=False,
    layout=widgets.Layout(width='100%', height='80px')
)

message_label = widgets.HTML(value='<b>Your Message:</b>')
message_input = widgets.Textarea(
    value='',
    placeholder='Type your message here...',
    description='',
    disabled=False,
    layout=widgets.Layout(width='100%', height='80px')
)

send_button = widgets.Button(
    description='Send',
    disabled=False,
    button_style='primary',
    tooltip='Send message',
    icon='paper-plane'
)

clear_button = widgets.Button(
    description='Clear Chat',
    disabled=False,
    button_style='warning',
    tooltip='Clear conversation',
    icon='trash'
)

model_dropdown = widgets.Dropdown(
    options=['gpt-4o-mini', 'gpt-3.5-turbo', 'gpt-4o'],
    value='gpt-4o-mini',
    description='Model:',
    disabled=False,
)

temperature_slider = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1.0,
    step=0.1,
    description='Temperature:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

conversation_output = widgets.HTML(
    value='<i>Conversation will appear here...</i>',
    placeholder='',
    description='',
)

status_output = widgets.HTML(
    value='',
    placeholder='',
    description='',
)

# Define button callbacks
def on_send_button_clicked(b):
    # Update system message if changed
    conversation[0]['content'] = system_input.value
    
    # Get user message
    user_message = message_input.value.strip()
    if not user_message:
        status_output.value = '<span style="color: red">Please enter a message</span>'
        return
    
    # Add user message to conversation
    conversation.append({
        'role': 'user',
        'content': user_message
    })
    
    # Clear input field
    message_input.value = ''
    
    # Update conversation display
    conversation_output.value = format_conversation(conversation)
    status_output.value = '<span style="color: blue">Thinking...</span>'
    
    # Get response from API
    response = send_message(
        conversation, 
        model=model_dropdown.value, 
        temperature=temperature_slider.value
    )
    
    # Add assistant response to conversation
    conversation.append({
        'role': 'assistant',
        'content': response
    })
    
    # Update conversation display
    conversation_output.value = format_conversation(conversation)
    status_output.value = ''

def on_clear_button_clicked(b):
    global conversation
    # Reset conversation to just the system message
    conversation = [
        {
            'role': 'system',
            'content': system_input.value
        }
    ]
    conversation_output.value = '<i>Conversation cleared</i>'
    status_output.value = ''

# Connect callbacks to buttons
send_button.on_click(on_send_button_clicked)
clear_button.on_click(on_clear_button_clicked)

# Handle Enter key in message input
def on_key_press(widget, event):
    if event.get('type') == 'keydown' and event.get('key') == 'Enter' and not event.get('shiftKey'):
        on_send_button_clicked(None)
        return True
    return False

message_input.observe(on_key_press, names=['_key_press'])

# Display the interface
display(system_label, system_input)
display(widgets.HBox([model_dropdown, temperature_slider]))
display(message_label, message_input)
display(widgets.HBox([send_button, clear_button]))
display(status_output)
display(conversation_output)

HTML(value='<b>System Message:</b>')

Textarea(value='You are ChillBuddy — a Gen Z-friendly, emotionally intelligent AI mental health counselor trai…

HTML(value='<b>Your Message:</b>')

Textarea(value='', layout=Layout(height='80px', width='100%'), placeholder='Type your message here...')

HTML(value='', placeholder='')

HTML(value='<i>Conversation will appear here...</i>', placeholder='')